In [0]:


spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import pandas as pd
pd.set_option('expand_frame_repr', False)
import boto3
from datetime import datetime
import pandas as pd
from pyspark.sql import functions as F
from applications.db_check_v1.common.constants import COUNTRY_CODE_MAPPING_BY_MARKET_CODE as COUNTRY_CODE_MAPPING
from conf.settings import *
from applications.db_check_v1.common.db_check_utils import query_df
import pandas as pd

daily_est_dsn =(
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_DAILY_EST_NAME,
        user=PG_DAILY_EST_ACCESS_ID,
        host=PG_DAILY_EST_HOSTS[0][0],
        password=PG_DAILY_EST_SECRET_KEY,
        port=PG_DAILY_EST_HOSTS[0][1]
    )
)


def get_date_list(start_date, end_date, freq="D"):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=start_date, end=end_date, freq=freq))]
    return date_list


def compare(date):
    sql = """
select distinct(app_id) from plproxy.execute_select_nestloop($proxy$ 
    select distinct(app_id)
    from aa.app_store_daily_estimate_{}
    where 
        date = '{}'
$proxy$) tbl (app_id BIGINT);
"""
    print datetime.today()
    d_app_list = []
    for device in [0,1,2]:
        db_df = query_df(daily_est_dsn, sql.format(device, date))
        d_app_list = d_app_list + list(db_df.app_id.unique())
    unified_df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}/".format(date)).select("app_id").distinct().toPandas()
    u_list = unified_df.app_id.unique()
    d_list = list(set(d_app_list))
    diff_list =  [x for x in  d_list if x not in u_list ]
    if len(diff_list) ==  0:
        return "{} PASS".format(date)
    else:
        print diff_list
        return "{} FAIL: {} {} {}".format(date, len(u_list), len(d_list), diff_list[0:5])
        
        
def write_log(strobj, s3obj):
    s3obj.put(Body=str(strobj))

s3 = boto3.resource('s3')
s3object = s3.Object('b2c-prod-data-pipeline-qa', 'tom/2019-2020/regression1.txt')
log = [] 

# 2010-07-04/   2020-02-21
date_list = ["2019-09-01"]# get_date_list("2019-02-19", "2019-05-31")
for date in date_list:
    temp_log = ""
    try:
        temp_log = compare(date)
        log.append(temp_log)
        print temp_log
    except Exception, e:
        temp_log = "{}: ERROR".format(date)
        print temp_log
        print e.message
    write_log(log, s3object)



In [0]:
%%sh

PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d dailyest -p 7432 << EOF 

select app_id,feed_id,store_id,estimate from plproxy.execute_select_nestloop(\$proxy\$ 
    select app_id,feed_id,store_id,estimate
    from aa.app_store_daily_estimate_1
    where 
        date = '2019-09-01' AND app_id in (985896290, 470074597, 497708025, 720245280, 540041963, 391329860, 829641544, 1148468242, 928368430, 859604306, 1046307415 )
\$proxy\$) tbl (app_id BIGINT,feed_id SMALLINT,store_id INT, estimate INT);

EOF


In [0]:
%%sh

PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d dailyest -p 7432 << EOF 

select app_id,feed_id,store_id,estimate from plproxy.execute_select_nestloop(\$proxy\$ 
    select app_id,feed_id,store_id,estimate
    from aa.app_store_daily_estimate_1
    where 
        date = '2019-09-01' AND app_id in (985896290, 470074597, 497708025, 720245280, 540041963, 391329860, 829641544, 1148468242, 928368430, 859604306, 1046307415 )
    limit 5
\$proxy\$) tbl (app_id BIGINT,feed_id SMALLINT,store_id INT, estimate INT);

EOF


In [0]:

from pyspark.sql import types as T
dates = ["2019-09-0" + str(i) for i in range(1, 8)]
def check(date):
    DF_path = "s3a://b2c-prod-dca-store-estimates/store_est/v_final/DAY/{date}/*/sbe_est_app/*/".format(date=date)
    EST_SCHEMA = T.StructType(
    [
        T.StructField("country_code", T.StringType(), True),
        T.StructField("date", T.StringType(), True),
        T.StructField("device_code", T.StringType(), True),
        T.StructField("granularity", T.StringType(), True),
        T.StructField("free_app_download", T.StringType(), True),  
        T.StructField("app_id", T.StringType(), True),
        T.StructField("paid_app_download", T.StringType(), True),
        T.StructField("revenue", T.StringType(), True),
        T.StructField("revenue_iap", T.StringType(), True)    ]
    )
    DF_df = spark.read.format("csv").option("header", "false").option("sep", "\t").load(DF_path, schema=EST_SCHEMA).cache()
    TECH_path = "s3a://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={date}".format(date=date)
    TECH_df = spark.read.parquet(TECH_path)
    print "data foundation ${date} distinct app id: ".format(date=date)
    print DF_df.select("app_id").distinct().count()
    print "tech team ${date} distinct app id: ".format(date=date)
    print(TECH_df.select("app_id").distinct().count())
for date in dates:
    check(date)



In [0]:

from pyspark.sql import types as T
import pandas as pd

EST_SCHEMA = T.StructType(
[
    T.StructField("country_code", T.StringType(), True),
    T.StructField("date", T.StringType(), True),
    T.StructField("device_code", T.StringType(), True),
    T.StructField("granularity", T.StringType(), True),
    T.StructField("free_app_download", T.StringType(), True),  
    T.StructField("app_id", T.StringType(), True),
    T.StructField("paid_app_download", T.StringType(), True),
    T.StructField("revenue", T.StringType(), True),
    T.StructField("revenue_iap", T.StringType(), True)    ]
)
    
def get_date_list(start_date, end_date, freq="D"):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=start_date, end=end_date, freq=freq))]
    return date_list

# date_list = get_date_list("2010-08-02", "2019-07-13")
date_list = get_date_list("2019-04-20", "2019-04-25")

for date in date_list:
    unifie_count = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}/".format(date)).select("app_id").distinct().count()

    raw_count = spark.read.option("delimiter", "\t").csv("s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/{}/*/sbe_est_app/*/".format(date)).select("_c5").distinct().count()

    if unifie_count!=raw_count:
        print "{}; Fail; {};{}".format(date, raw_count, unifie_count)
    else:
        print "{}; Pass; {};{}".format(date, raw_count, unifie_count)

In [0]:

import pandas as pd

def get_date_list(start_date, end_date, freq="D"):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=start_date, end=end_date, freq=freq))]
    return date_list

date_list = get_date_list("2019-07-14", "2020-02-24")

for date in date_list:
    df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}/".format(date))
    unifie_count = df.select("app_id").distinct().count()

    df = spark.read.parquet("s3://b2c-prod-dca-store-estimates/store_estv2/APP_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date={}/".format(date))
    raw_count= df.select("id").distinct().count()
    
    if unifie_count!=raw_count:
        print "{}; Fail; {};{}".format(date, raw_count, unifie_count)
    else:
        print "{}; Pass".format(date)

In [0]:

date="2019-07-14"
df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}/".format(date))
df.select("app_id").distinct().write.parquet("s3://b2c-prod-data-pipeline-qa/tom/store/unified/date={}/".format(date))

In [0]:
%%sh

aws s3 ls s3://b2c-prod-data-pipeline-qa/tom/store/unified/date={}/app_ids.parquet/


In [0]:


spark.read.option("delimiter", "\t").csv("s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/2010-07-04/*/sbe_est_app/*/*.csv.gz").select("_c5").distinct().count()



In [0]:
%%sh

aws s3 ls s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/2019-01-01/android/sbe_est_app/1/


In [0]:
%%sh

aws s3 ls  s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date=2019-04-20/device_code=ios-all/


In [0]:
%%sh%%sh

PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d dailyest -p 7432 << EOF 

select device_id from plproxy.execute_select_nestloop(\$proxy\$ 
    select distinct(device_id)
    from pp.pub_store_daily_estimate_0_1_202001
    where 
        date='2020-01-01'
\$proxy\$) tbl (device_id SMALLINT);

EOF



In [0]:



spark.read.parquet
